# Actividad Big Data
### Gerardo Peña Pérez
### A01701474


In [1]:
# Bibliotecas para poder trabajar con Spark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz  
# Configuración de Spark con Python
!pip install -q findspark
!pip install pyspark

# Estableciendo variable de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

# Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Building dependency tree       
Reading state info

'/content/spark-3.2.2-bin-hadoop3.2'

In [2]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [3]:
# Creamos la sesión de spark
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.appName('ActividadBigData').getOrCreate()
spark_session

In [4]:
# Cargamos el data set que se usará
# En este caso elegí un dataset sobre cancer de mama
# Intentaré predecir si el tumor es benigno o maligno basandome en distintas características del tumor
dataset = spark_session.read.csv('sample_data/breast-cancer-wisconsin (1).csv', header=True, inferSchema=True)
dataset.show()

+------------------+---------------+-----------------------+------------------------+-----------------+---------------------------+-----------+---------------+---------------+-------+-----+
|Sample_code_number|Clump_Thickness|Uniformity_of_Cell_Size|Uniformity_of_Cell_Shape|Marginal_Adhesion|Single_Epithelial_Cell_Size|Bare_Nuclei|Bland_Chromatin|Normal_Nucleoli|Mitoses|Class|
+------------------+---------------+-----------------------+------------------------+-----------------+---------------------------+-----------+---------------+---------------+-------+-----+
|           1000025|              5|                      1|                       1|                1|                          2|          1|              3|              1|      1|    2|
|           1002945|              5|                      4|                       4|                5|                          7|         10|              3|              2|      1|    2|
|           1015425|              3|              

In [5]:
# Quiero ver si todos los tipos de datos son correctos
dataset.printSchema()

root
 |-- Sample_code_number: integer (nullable = true)
 |-- Clump_Thickness: integer (nullable = true)
 |-- Uniformity_of_Cell_Size: integer (nullable = true)
 |-- Uniformity_of_Cell_Shape: integer (nullable = true)
 |-- Marginal_Adhesion: integer (nullable = true)
 |-- Single_Epithelial_Cell_Size: integer (nullable = true)
 |-- Bare_Nuclei: string (nullable = true)
 |-- Bland_Chromatin: integer (nullable = true)
 |-- Normal_Nucleoli: integer (nullable = true)
 |-- Mitoses: integer (nullable = true)
 |-- Class: integer (nullable = true)



In [6]:
# El atributo Sample_code_number es solo un identificador, así que lo puedo quitar,al igual que otras columnas innecesarias
dataset = dataset.drop("Sample_code_number")
dataset = dataset.drop("Bare_Nuclei")

In [7]:
# Asignamos como indice a la columna con el resultado (class=2 -> benigno) (class=4 -> maligno)
my_index = StringIndexer(inputCol='Class', outputCol='classNew')
fitted = my_index.fit(dataset)
new_df = fitted.transform(dataset)
new_df = new_df.drop("Class")

# dividimos entre datos de entrenamiento y datos de testeo,asignando 70% y 30% respectivamente

(data_train, data_test) = new_df.randomSplit([0.7, 0.3], seed=17)

In [8]:
cols = ["Clump_Thickness",
        "Uniformity_of_Cell_Size", "Uniformity_of_Cell_Shape", 
        "Marginal_Adhesion", "Single_Epithelial_Cell_Size",
        #"Bare_Nuclei",
         "Bland_Chromatin", 
        "Normal_Nucleoli", "Mitoses"]

assembler = VectorAssembler(
    inputCols=cols,
    outputCol="features")

transformed_train = assembler.transform(data_train)
transformed_test = assembler.transform(data_test)
transformed_train = transformed_train.drop(*cols)
transformed_test = transformed_test.drop(*cols)
transformed_train.show()

+--------+--------------------+
|classNew|            features|
+--------+--------------------+
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
|     0.0|[1.0,1.0,1.0,1.0,...|
+--------+--------------------+
only showing top 20 rows



In [9]:
from pyspark.mllib import linalg as mllib_linalg
from pyspark.ml import linalg as ml_linalg

def as_old(v):
    if isinstance(v, ml_linalg.SparseVector):
        return mllib_linalg.SparseVector(v.size, v.indices, v.values)
    if isinstance(v, ml_linalg.DenseVector):
        return mllib_linalg.DenseVector(v.values)
    raise ValueError("Unsupported type {0}".format(type(v)))

In [10]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col

train_dataset = (transformed_train.select(col("classNew").alias("label"), col("features"))\
                  .rdd.map(lambda row: LabeledPoint(row.label, as_old(row.features))))


test_dataset = (transformed_test.select(col("classNew").alias("label"), col("features"))\
                  .rdd.map(lambda row: LabeledPoint(row.label, as_old(row.features))))

In [11]:
# Como el problema con el que nos encontramos es de clasificación aplicaremos un decision tree con MLlib de pyspark
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.tree import DecisionTreeModel

numClasses = 2
categoricalFeaturesInfo = {}
impurity = "gini"

# transformamos el dataframe a un rdd y poder usar MLlib
model = DecisionTree.trainClassifier(train_dataset, numClasses, 
                                     categoricalFeaturesInfo, impurity)

In [12]:
predictions = model.predict(test_dataset.map(lambda x: x.features))
labelsAndPredictions = test_dataset.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda lp: lp[0] != lp[1]).count() / float(test_dataset.count())
print('Test Error = ' + str(testErr))
print('Tree model:')
print(model.toDebugString())

Test Error = 0.136986301369863
Tree model:
DecisionTreeModel classifier of depth 5 with 31 nodes
  If (feature 1 <= 2.5)
   If (feature 0 <= 6.5)
    If (feature 4 <= 5.5)
     If (feature 6 <= 3.5)
      Predict: 0.0
     Else (feature 6 > 3.5)
      If (feature 0 <= 4.5)
       Predict: 0.0
      Else (feature 0 > 4.5)
       Predict: 1.0
    Else (feature 4 > 5.5)
     If (feature 0 <= 3.5)
      Predict: 0.0
     Else (feature 0 > 3.5)
      Predict: 1.0
   Else (feature 0 > 6.5)
    If (feature 5 <= 2.5)
     Predict: 0.0
    Else (feature 5 > 2.5)
     Predict: 1.0
  Else (feature 1 > 2.5)
   If (feature 5 <= 3.5)
    If (feature 0 <= 6.5)
     If (feature 3 <= 2.5)
      Predict: 0.0
     Else (feature 3 > 2.5)
      If (feature 2 <= 2.5)
       Predict: 0.0
      Else (feature 2 > 2.5)
       Predict: 1.0
    Else (feature 0 > 6.5)
     If (feature 2 <= 3.5)
      Predict: 0.0
     Else (feature 2 > 3.5)
      Predict: 1.0
   Else (feature 5 > 3.5)
    If (feature 5 <= 4.5)
   